----------------

# Photometry Observations of Didymos Asteroid (Part 4 of 4)

# Creating Composite Asteroid Light Curves to Determine Rotation and Mutual Orbital Period

## By Arushi Nath, Founder: MonitorMyPlanet.com


-----------------------------

#### This Fourth Notebook will take you through the steps of finding the rotational period of Didymos and the orbital period of Dimorphos. It will use the light curve created in part 3. It is preferable to have multiple nights of observations at this step to get more accurate results.

-------------------------------

### 1. Setting up the Enviornment 

In [ ]:
### Importing Required Libraries ###
import csv
import matplotlib.pyplot as plt
import juliandate as jd
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import math
from sklearn import preprocessing

In [ ]:
### Importing Light Curve of Multiple Observations ###
mag = []
JD2 = []
JD = []
Error = []
Color = []
with open(r"(ADD YOUR FOLDER DIRECTORY HERE)", newline='') as f:
  reader = csv.reader(f)
  for row in reader:
    print(row)
    
    mag.append((row[1]))
    
    JD.append((row[0]))
    
    Error.append((row[2]))
    
    #Color.append(row[3])
    
mag = mag[1:]
mag = [eval(i) for i in mag]

JD = JD[1:]

JD = [eval(i) for i in JD]

Error = Error[1:]
Error = [eval(i) for i in Error]

#Color = Color[1:]
#Color = [eval(i) for i in Color]

In [ ]:
### Converting Julian Date into Hours from Start ###

def UnixTimeFromJulianDate(jd):
    return ((jd-2440587.5)*86400)/3600
s = (UnixTimeFromJulianDate(JD[0]))

hours = []
for i in JD:
    hours.append(UnixTimeFromJulianDate(i) - s)

In [ ]:
### Scatter all Observations of Didymos ###
plt.figure(figsize=(5, 4))
plt.scatter(JD,mag, marker = "v")

plt.title("Total Observations - Pre-Impact")
plt.xlabel("Hours")
plt.ylabel("Magnitude")

### 2. Finding Primary Period (Rotation of Didymos)

In [ ]:
### Creating Function to Create Composite Light Curves for any period ###
def period2curve(period):
    p_time = []
    for i in range(len(hours)):
        p_time.append(hours[i]%period)
    return p_time

In [ ]:
### Estimating the Periodicity to be in Between 2 and 5 Hours. Creating 500 Intervals between those Numbers ###
rotations = []
n = np.linspace(2, 5, 5000)
for i in tqdm(n):
    rotations.append(period2curve(i))

In [ ]:
### Finding the Root Mean Square Error of Composite Light Curves created with Periodicity Between 2 and 5 Hours ###

num = 5000
RMSE = []
line = []
nu = []
nu2 = []
ln = []
test = []

for i in tqdm(range(len(rotations))):
   
    nu2.append(i)
    duplicate = rotations[i]
    norm = preprocessing.normalize([np.array(rotations[i])])
    
    df = pd.DataFrame({'x': rotations[i],
                'y': mag})
        
    model5 = np.poly1d(np.polyfit(df.x, df.y, 13))

    polyline = np.linspace(0, rotations[i], num)
    for t in range(len(norm[0])):
        line.append(model5(rotations[i][t]))
    ln.append(line)
    
    MSE = np.square(np.subtract(mag,line)).mean() 
 
    RMSE.append(math.sqrt(MSE))
    test.append(line)
    plt.figure(figsize=(5, 4))
    
    
    plt.errorbar(rotations[i],mag, fmt="o", marker = "X")

    
    t = "Period (hours): " + str(max(rotations[i]))
    print(max(rotations[i]),math.sqrt(MSE))
    plt.title(t)
    plt.ylabel("Magnitude")
    plt.xlabel("Phase")
    #plt.ylim(16.4,15.6)
    
    plt.scatter(rotations[i], line, color='red', s = 12, zorder=5)
    plt.legend()
    plt.show()
    line = []
    

In [ ]:
### Plotting RMSE for all Periods ###
plt.figure(figsize=(20, 10))
plt.plot(n,RMSE)

In [ ]:
### Finding the 5 Periods with the Smallest RMSE.
t_periods = []
t_plots = []
t_test = []
ln2 = []
RMSE_mod = RMSE.copy()
RMSE_max = []
for s in range(5):
    RMSE_max.append(min(RMSE_mod))
    for i in range(len(RMSE)):
        try:
            if RMSE_mod[i] == min(RMSE_mod):
                t_periods.append(n[i])
                t_plots.append(i)
                t_test.append(test[i])
                ln2.append(ln[i])
                
        except:
            pass
    
    RMSE_mod.remove(min(RMSE_mod))
    

In [ ]:
### Plotting Top 5 Possible Periods. The First one should be 2.26, the True Rotational Period of Didymos ###
for i in range(len(t_plots)):
    title = "Period: " +  str(t_periods[i])

    print(RMSE_max[i])
    plt.figure(figsize=(6, 4))
    #plt.scatter(rotations[t_plots[i]],mag, s = 10)
    plt.scatter(rotations[t_plots[i]], ln2[i], color='red', s = 12, zorder=5)
    plt.title(title)
    plt.xlabel("Phase")
    plt.ylabel("Magnitude")
    #plt.ylim(16.4,15.6)

    plt.errorbar(rotations[t_plots[i]],mag, fmt="o", marker = "X")
    
    
    plt.legend()
    plt.show()

### 3. Finding Secondary Period (Dimorphos Orbit)

In [ ]:
### Substracting Rotational Period from the Original Observations. The Orbit of Dimorphos should be what is Left Out ###
substract  = []
for i in range(len(t_test[0])):  
    substract.append(mag[i]-t_test[0][i])

In [ ]:
### Estimating the Periodicity of Dimorphos' Orbit to be in Between 8 and 16 Hours, and creating 5000 Intervals in Between ###
orbits = []
w = []
n = np.linspace(8, 16, 5000)
for i in tqdm(range(len(n))):
    orbits.append(period2curve(n[i]))
    w.append(2+(i/714.285714))

In [ ]:
### Finding the Root Mean Square Error of Composite Light Curves created with Periodicity Between 8 and 16 Hours ###

RMSE_o = []
line_o = []
nu_o = []
c = 0
for i in tqdm(orbits):
    nu_o.append(w[c])
    duplicate_o = i.copy()
    norm_o = preprocessing.normalize([np.array(i)])
    
    df = pd.DataFrame({'x': i,
                'y': substract})
        
    model5_o = np.poly1d(np.polyfit(df.x, df.y, 13))

    for t in range(len(norm_o[0])):
        line_o.append(model5_o(i[t]))
        

    plt.scatter(i,substract)
    print(len(i), len(line_o), len(substract))
    

    plt.errorbar(i,substract, fmt="o", marker = "X")
    
    plt.scatter(i, line_o, color='red', zorder = 5)
    
    
    MSE_o = np.square(np.subtract(substract,line_o)).mean() 
 
    RMSE_o.append(math.sqrt(MSE_o))
    
    print(math.sqrt(MSE_o))
    
    title = "period: " + str(w[c])
    plt.title(title)
    plt.xlabel("Phase")
    plt.ylabel("Magnitude")
    plt.ylim(plt.ylim()[::-1])
    plt.ylim(-0.3,0.3)
    plt.show()
    
    line_o = []
    c+=1

In [ ]:
### Finding the Top 5 Periods with the Least RMSE ###
t_periods2 = []
t_plots2 = []

RMSE_mod2 = RMSE_o.copy()
RMSE_max2 = []
for s in range(5):
    RMSE_max2.append(min(RMSE_mod2))
    for i in range(len(RMSE_mod2)):
    
        if RMSE_mod2[i] == min(RMSE_mod2):
            t_periods2.append(nu_o[i])
            t_plots2.append(i)
               
                
        
    
    RMSE_mod2.remove(min(RMSE_mod2))
    

In [ ]:
### Plotting the Top 5 Periods. The First one should be either 11.91 or 11.38, Depending if your Observations and Before of After the Impact ###
for i in range(len(t_plots2)):
    title = "Period", ((t_periods2[i]-8)/5)+8, ", RMSE (x0.1 mag):", RMSE_max2[i]*100
    plt.figure(figsize=(20, 10))
    plt.scatter(orbits[t_plots2[i]],mag)
    plt.title(title)
    plt.ylim(plt.ylim()[::-1])
    plt.show()